<a href="https://colab.research.google.com/github/leonardoub/SCRIPT_PALERMO/blob/master/prova_hyperparameter_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install -U keras-tuner

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [0]:
!rm -r my_dir

#Load data

In [4]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
train_dataset_path = '/gdrive/My Drive/AIM_PA/database_training2.csv'
test_dataset_path = '/gdrive/My Drive/AIM_PA/database_nostro_without_nan.csv'

In [0]:
df_train = pd.read_csv(train_dataset_path)
df_test = pd.read_csv(test_dataset_path)

In [0]:
df_train.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_test.rename(columns={'Survival.time (months)':'Surv_time_months'}, inplace=True)

In [0]:
df_train.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)
df_test.rename(columns={'Overall.Stage':'Overall_Stage'}, inplace=True)

In [0]:
train_data = df_train.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
test_data = df_test.drop(['Histology', 'Surv_time_months', 'OS', 'deadstatus.event','Overall_Stage'], axis=1)

In [0]:
train_labels = df_train.Histology

In [0]:
test_labels = df_test.Histology

##Z score dei dati

In [0]:
mean = train_data.mean(axis=0)
train_data_stand = train_data - mean
std = train_data.std(axis=0)
train_data_stand /= std

In [0]:
test_data_stand = test_data - mean
test_data_stand /= std

##Vettorizzare i label

In [0]:
word_index={'adenocarcinoma':0, 'large cell':1, 'squamous cell carcinoma':2}

In [0]:
train_labels_dec = [word_index[label] for label in train_labels]

In [0]:
test_labels_dec = [word_index[label] for label in test_labels]

In [19]:
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [0]:
one_hot_train_labels = to_categorical(train_labels_dec)
one_hot_test_labels = to_categorical(test_labels_dec)

#PCA

In [0]:
from sklearn.decomposition import PCA

In [0]:
pca = PCA(n_components=0.9, svd_solver='full')

In [23]:
pca.fit(train_data_stand)

PCA(copy=True, iterated_power='auto', n_components=0.9, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [0]:
train_data_stand_pca = pca.transform(train_data_stand)
test_data_stand_pca = pca.transform(test_data_stand)

In [25]:
train_data_stand_pca.shape

(131, 9)

#Building Network

In [0]:
from tensorflow import keras

In [0]:
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence

In [0]:
from tensorflow.keras import layers

In [0]:
from tensorflow.keras import regularizers

In [0]:
from tensorflow.keras.optimizers import SGD

In [0]:
from tensorflow.keras.optimizers import Adam

In [0]:
from kerastuner.tuners import RandomSearch, Hyperband

In [0]:
def build_model(hp):
  model = keras.models.Sequential()
  activation = hp.Choice('activation', ['softmax', 'softplus', 'softsign',
                              'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear'])
  
  weight_init = hp.Choice('weight_init', 
                            ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'])
  
  drop_rate = hp.Choice('drop_rate', [0.0, 0.1, 0.2, 0.3,
                              0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
  
  model.add(layers.Dense(units=(hp.Int('units', min_value=3, max_value=8, step=1)), 
                         activation=activation, kernel_initializer=weight_init, input_shape=(9,)))
  
  model.add(layers.Dropout(rate=drop_rate))

  model.add(layers.Dense(3, activation='softmax'))
  #sgd_1 = SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)
 
  #optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])

  lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
  #momentum = hp.Choice('momentum', [0.0, 0.2, 0.4, 0.6, 0.8, 0.9])
  
  model.compile(optimizer=keras.optimizers.Adam(lr), 
                loss='categorical_crossentropy', metrics=['accuracy'])
  return model

#Stratified k-fold

This cross-validation object is a variation of KFold that returns stratified folds. The folds are made by preserving the percentage of samples for each class.

In [0]:
from sklearn.model_selection import StratifiedKFold

In [35]:
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
skf.get_n_splits(train_data_stand_pca, train_labels_dec)

3

In [36]:
for train_index, test_index in skf.split(train_data_stand_pca, train_labels_dec):
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [  0   1   4   5   8   9  11  12  14  15  16  17  19  20  22  23  24  25
  27  29  30  33  34  36  37  38  39  40  41  42  44  45  46  48  51  52
  53  56  57  58  59  60  62  63  65  66  67  69  72  76  77  78  79  80
  81  83  84  85  87  88  89  90  92  96  97  98 100 101 102 103 104 105
 107 109 110 111 113 115 117 120 121 122 124 125 127 128] TEST: [  2   3   6   7  10  13  18  21  26  28  31  32  35  43  47  49  50  54
  55  61  64  68  70  71  73  74  75  82  86  91  93  94  95  99 106 108
 112 114 116 118 119 123 126 129 130]
TRAIN: [  2   3   5   6   7   8   9  10  11  12  13  18  20  21  25  26  27  28
  29  30  31  32  34  35  36  38  39  43  44  45  46  47  48  49  50  53
  54  55  57  58  61  63  64  65  66  68  70  71  73  74  75  76  78  82
  84  85  86  87  90  91  92  93  94  95  96  99 100 101 102 105 106 108
 109 111 112 114 115 116 118 119 122 123 124 125 126 127 129 130] TEST: [  0   1   4  14  15  16  17  19  22  23  24  33  37  40  41  42  51  52
  56  59 

In [37]:
train_labels_dec[125]

2

In [0]:
from keras.utils import to_categorical


In [0]:
#for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
#  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
#  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])

#  val_data = np.array([train_data_stand_pca[i] for i in val_index])
#  val_targets = np.array([train_labels_dec[i] for i in val_index])

#  one_hot_partial_train_targets = to_categorical(partial_train_targets)
#  one_hot_val_targets = to_categorical(val_targets)

#  model = build_model()
#  model.fit(partial_train_data, one_hot_partial_train_targets, epochs = num_epochs, batch_size=1)

#  val_loss, val_accuracy = model.evaluate(val_data, one_hot_val_targets)
#  all_scores.append(val_accuracy)
#I parametri per la valutazione vengono calcolati una volta per ogni k-fold, per ogni set di validazione, quindi k volte

C'è un problema: keras.utils.to_categorical produces a one-hot encoded class vector, i.e. the multilabel-indicator mentioned in the error message. StratifiedKFold is not designed to work with such input; i.e. your y must be a 1-D array of your class labels.
Essentially, what you have to do is simply to invert the order of the operations: split first (using your intial y_train), and convert to_categorical afterwards.

###Keras tuner RandomSearch

In [40]:
num_epochs = 30
all_acc_histories = []
all_loss_histories = []
all_val_acc_histories = []
all_val_loss_histories = []

for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

 
  tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=20, 
                       executions_per_trial=5, directory='/content/my_dir', project_name='RandomSearch')
  
  tuner.search_space_summary()

  tuner.search(partial_train_data, one_hot_partial_train_targets, validation_data=(val_data, one_hot_val_targets), 
                      epochs=num_epochs, batch_size=1)
  

#  acc_history = history.history['acc']
#  all_acc_histories.append(acc_history)

#  loss_history = history.history['loss']
#  all_loss_histories.append(loss_history)

#  acc_val_history = history.history['val_acc']
#  all_val_acc_histories.append(acc_val_history)

#  loss_val_history = history.history['val_loss']
#  all_val_loss_histories.append(loss_val_history)
  

#I parametri per la valutazione vengono calcolati per ogni epoca, quindi num_epochs volte. 
#Il tutto viene ripetuto un numero di volte pari a n_splits.
#Si ottiene una lista con n_splits elementi ciascuno dei quali è una lista lunga num_epochs,
#ogni elemento può essere uno fra questi: dict_keys(['val_loss', 'val_acc', 'loss', 'acc']) 

Train on 86 samples, validate on 45 samples
Epoch 1/30
86/86 [==============================] - 1s 7ms/sample - loss: 1.2082 - accuracy: 0.4070 - val_loss: 0.9957 - val_accuracy: 0.4000
Epoch 2/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0585 - accuracy: 0.4419 - val_loss: 1.0270 - val_accuracy: 0.4000
Epoch 3/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0116 - accuracy: 0.5581 - val_loss: 0.9919 - val_accuracy: 0.5111
Epoch 4/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.9163 - accuracy: 0.5233 - val_loss: 0.9782 - val_accuracy: 0.5111
Epoch 5/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.9067 - accuracy: 0.5814 - val_loss: 0.9954 - val_accuracy: 0.5111
Epoch 6/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.9573 - accuracy: 0.5581 - val_loss: 1.0072 - val_accuracy: 0.4444
Epoch 7/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0063 - accuracy: 0.5233 - va

Train on 86 samples, validate on 45 samples
Epoch 1/30
86/86 [==============================] - 1s 6ms/sample - loss: 3.3129 - accuracy: 0.2791 - val_loss: 1.1423 - val_accuracy: 0.4889
Epoch 2/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.5823 - accuracy: 0.4070 - val_loss: 0.9846 - val_accuracy: 0.4444
Epoch 3/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.1714 - accuracy: 0.5116 - val_loss: 0.9823 - val_accuracy: 0.4222
Epoch 4/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0789 - accuracy: 0.4070 - val_loss: 0.9734 - val_accuracy: 0.4889
Epoch 5/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0512 - accuracy: 0.4651 - val_loss: 0.9910 - val_accuracy: 0.4889
Epoch 6/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.9816 - accuracy: 0.4651 - val_loss: 0.9766 - val_accuracy: 0.4889
Epoch 7/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0493 - accuracy: 0.4419 - va

Train on 86 samples, validate on 45 samples
Epoch 1/30
86/86 [==============================] - 0s 6ms/sample - loss: 6.0009 - accuracy: 0.2791 - val_loss: 4.1781 - val_accuracy: 0.3333
Epoch 2/30
86/86 [==============================] - 0s 2ms/sample - loss: 6.8965 - accuracy: 0.2326 - val_loss: 4.0886 - val_accuracy: 0.3333
Epoch 3/30
86/86 [==============================] - 0s 2ms/sample - loss: 6.0397 - accuracy: 0.2326 - val_loss: 4.0097 - val_accuracy: 0.3333
Epoch 4/30
86/86 [==============================] - 0s 2ms/sample - loss: 6.5083 - accuracy: 0.2558 - val_loss: 3.9334 - val_accuracy: 0.3333
Epoch 5/30
86/86 [==============================] - 0s 2ms/sample - loss: 5.7427 - accuracy: 0.2442 - val_loss: 3.8624 - val_accuracy: 0.3333
Epoch 6/30
86/86 [==============================] - 0s 2ms/sample - loss: 6.1918 - accuracy: 0.2093 - val_loss: 3.7843 - val_accuracy: 0.3333
Epoch 7/30
86/86 [==============================] - 0s 2ms/sample - loss: 5.8050 - accuracy: 0.2674 - va

Train on 86 samples, validate on 45 samples
Epoch 1/30
86/86 [==============================] - 1s 6ms/sample - loss: 2.5405 - accuracy: 0.4070 - val_loss: 1.5703 - val_accuracy: 0.4667
Epoch 2/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.7884 - accuracy: 0.4070 - val_loss: 1.0616 - val_accuracy: 0.4889
Epoch 3/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.1056 - accuracy: 0.5930 - val_loss: 1.0036 - val_accuracy: 0.5333
Epoch 4/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.9381 - accuracy: 0.6163 - val_loss: 0.9852 - val_accuracy: 0.5778
Epoch 5/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.9780 - accuracy: 0.5930 - val_loss: 0.9854 - val_accuracy: 0.5333
Epoch 6/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0414 - accuracy: 0.5465 - val_loss: 0.9787 - val_accuracy: 0.5333
Epoch 7/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0249 - accuracy: 0.4651 - va

Train on 86 samples, validate on 45 samples
Epoch 1/30
86/86 [==============================] - 1s 6ms/sample - loss: 1.2190 - accuracy: 0.3140 - val_loss: 1.1698 - val_accuracy: 0.3556
Epoch 2/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.2438 - accuracy: 0.3256 - val_loss: 1.1657 - val_accuracy: 0.3778
Epoch 3/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.1839 - accuracy: 0.3721 - val_loss: 1.1615 - val_accuracy: 0.3778
Epoch 4/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.2874 - accuracy: 0.3023 - val_loss: 1.1572 - val_accuracy: 0.3556
Epoch 5/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.2794 - accuracy: 0.3140 - val_loss: 1.1526 - val_accuracy: 0.3778
Epoch 6/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.2698 - accuracy: 0.3023 - val_loss: 1.1488 - val_accuracy: 0.4000
Epoch 7/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.1918 - accuracy: 0.3837 - va

...

Train on 86 samples, validate on 45 samples
Epoch 1/30
86/86 [==============================] - 1s 6ms/sample - loss: 1.4130 - accuracy: 0.3140 - val_loss: 1.0499 - val_accuracy: 0.4889
Epoch 2/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.1556 - accuracy: 0.4884 - val_loss: 1.0110 - val_accuracy: 0.5111
Epoch 3/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.8972 - accuracy: 0.5814 - val_loss: 1.0069 - val_accuracy: 0.5333
Epoch 4/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.9572 - accuracy: 0.5698 - val_loss: 1.0116 - val_accuracy: 0.5111
Epoch 5/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.8838 - accuracy: 0.6163 - val_loss: 1.0010 - val_accuracy: 0.5333
Epoch 6/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.8481 - accuracy: 0.6744 - val_loss: 1.0141 - val_accuracy: 0.5333
Epoch 7/30
86/86 [==============================] - 0s 2ms/sample - loss: 0.8391 - accuracy: 0.6860 - va

Train on 86 samples, validate on 45 samples
Epoch 1/30
86/86 [==============================] - 1s 6ms/sample - loss: 1.1223 - accuracy: 0.3488 - val_loss: 1.1092 - val_accuracy: 0.3333
Epoch 2/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0829 - accuracy: 0.4419 - val_loss: 1.1074 - val_accuracy: 0.3333
Epoch 3/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0866 - accuracy: 0.4419 - val_loss: 1.1054 - val_accuracy: 0.3556
Epoch 4/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0975 - accuracy: 0.4302 - val_loss: 1.1038 - val_accuracy: 0.3556
Epoch 5/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0514 - accuracy: 0.4302 - val_loss: 1.1020 - val_accuracy: 0.3556
Epoch 6/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.0662 - accuracy: 0.3372 - val_loss: 1.1005 - val_accuracy: 0.3556
Epoch 7/30
86/86 [==============================] - 0s 2ms/sample - loss: 1.1172 - accuracy: 0.3953 - va

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Reloading Oracle from /content/my_dir/RandomSearch/oracle.json
INFO:tensorflow:Reloading Tuner from /content/my_dir/RandomSearch/tuner0.json


INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Reloading Oracle from /content/my_dir/RandomSearch/oracle.json
INFO:tensorflow:Reloading Tuner from /content/my_dir/RandomSearch/tuner0.json


INFO:tensorflow:Oracle triggered exit


In [41]:
tuner.search_space_summary()


In [42]:
tuner.results_summary()

In [43]:
tuner.get_best_hyperparameters()

In [44]:
tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

{'activation': 'linear',
 'drop_rate': 0.0,
 'learning_rate': 0.001,
 'units': 8,
 'weight_init': 'glorot_uniform'}

In [0]:
best_model = tuner.get_best_models()[0]

###Keras tuner Hyperband

In [46]:
num_epochs = 50
all_acc_histories = []
all_loss_histories = []
all_val_acc_histories = []
all_val_loss_histories = []

for train_index, val_index in skf.split(train_data_stand_pca, train_labels_dec):
 
  partial_train_data = np.array([train_data_stand_pca[i] for i in train_index])
  partial_train_targets = np.array([train_labels_dec[i] for i in train_index])
  
  val_data = np.array([train_data_stand_pca[i] for i in val_index])
  val_targets = np.array([train_labels_dec[i] for i in val_index])

  one_hot_partial_train_targets = to_categorical(partial_train_targets)
  one_hot_val_targets = to_categorical(val_targets)

 
  tuner_Hb = Hyperband(build_model, objective='val_accuracy', factor=3, 
                       max_epochs=10, directory='/content/my_dir', project_name='Hyperband')
  
  tuner_Hb.search_space_summary()

  tuner_Hb.search(partial_train_data, one_hot_partial_train_targets, 
                  validation_data=(val_data, one_hot_val_targets))
  

#  acc_history = history.history['acc']
#  all_acc_histories.append(acc_history)

#  loss_history = history.history['loss']
#  all_loss_histories.append(loss_history)

#  acc_val_history = history.history['val_acc']
#  all_val_acc_histories.append(acc_val_history)

#  loss_val_history = history.history['val_loss']
#  all_val_loss_histories.append(loss_val_history)
  

#I parametri per la valutazione vengono calcolati per ogni epoca, quindi num_epochs volte. 
#Il tutto viene ripetuto un numero di volte pari a n_splits.
#Si ottiene una lista con n_splits elementi ciascuno dei quali è una lista lunga num_epochs,
#ogni elemento può essere uno fra questi: dict_keys(['val_loss', 'val_acc', 'loss', 'acc']) 

Train on 86 samples, validate on 45 samples
Epoch 1/2
86/86 [==============================] - 0s 4ms/sample - loss: 2.9509 - accuracy: 0.2907 - val_loss: 2.2734 - val_accuracy: 0.2889
Epoch 2/2
86/86 [==============================] - 0s 253us/sample - loss: 2.8632 - accuracy: 0.3605 - val_loss: 2.2362 - val_accuracy: 0.2889


Train on 86 samples, validate on 45 samples
Epoch 1/2
86/86 [==============================] - 0s 5ms/sample - loss: 1.4920 - accuracy: 0.1860 - val_loss: 1.2057 - val_accuracy: 0.2889
Epoch 2/2
86/86 [==============================] - 0s 354us/sample - loss: 1.3468 - accuracy: 0.3023 - val_loss: 1.1773 - val_accuracy: 0.3111


Train on 86 samples, validate on 45 samples
Epoch 1/2
86/86 [==============================] - 0s 4ms/sample - loss: 3.0440 - accuracy: 0.3488 - val_loss: 2.3584 - val_accuracy: 0.3778
Epoch 2/2
86/86 [==============================] - 0s 243us/sample - loss: 2.9356 - accuracy: 0.3488 - val_loss: 2.3310 - val_accuracy: 0.3778


Train on 86 samples, validate on 45 samples
Epoch 1/2
86/86 [==============================] - 0s 4ms/sample - loss: 5.8144 - accuracy: 0.3488 - val_loss: 3.9828 - val_accuracy: 0.3556
Epoch 2/2
86/86 [==============================] - 0s 297us/sample - loss: 8.2231 - accuracy: 0.3837 - val_loss: 3.9791 - val_accuracy: 0.3556


Train on 86 samples, validate on 45 samples
Epoch 1/2
86/86 [==============================] - 0s 4ms/sample - loss: 2.1534 - accuracy: 0.3023 - val_loss: 1.8396 - val_accuracy: 0.3556
Epoch 2/2
86/86 [==============================] - 0s 294us/sample - loss: 2.2065 - accuracy: 0.3140 - val_loss: 1.8374 - val_accuracy: 0.3556


...

Train on 86 samples, validate on 45 samples
Epoch 1/10
86/86 [==============================] - 0s 5ms/sample - loss: 1.1713 - accuracy: 0.3488 - val_loss: 1.0456 - val_accuracy: 0.4222
Epoch 2/10
86/86 [==============================] - 0s 369us/sample - loss: 1.0894 - accuracy: 0.3837 - val_loss: 1.0178 - val_accuracy: 0.4667
Epoch 3/10
86/86 [==============================] - 0s 258us/sample - loss: 0.9898 - accuracy: 0.5000 - val_loss: 1.0010 - val_accuracy: 0.4222
Epoch 4/10
86/86 [==============================] - 0s 424us/sample - loss: 0.9855 - accuracy: 0.5465 - val_loss: 0.9924 - val_accuracy: 0.5111
Epoch 5/10
86/86 [==============================] - 0s 287us/sample - loss: 0.9614 - accuracy: 0.5349 - val_loss: 0.9905 - val_accuracy: 0.5111
Epoch 6/10
86/86 [==============================] - 0s 253us/sample - loss: 1.0008 - accuracy: 0.6163 - val_loss: 0.9886 - val_accuracy: 0.5111
Epoch 7/10
86/86 [==============================] - 0s 367us/sample - loss: 0.9413 - accuracy:

Train on 86 samples, validate on 45 samples
Epoch 1/10
86/86 [==============================] - 0s 4ms/sample - loss: 1.0108 - accuracy: 0.4884 - val_loss: 0.9585 - val_accuracy: 0.4667
Epoch 2/10
86/86 [==============================] - 0s 307us/sample - loss: 1.0105 - accuracy: 0.4767 - val_loss: 0.9584 - val_accuracy: 0.4667
Epoch 3/10
86/86 [==============================] - 0s 301us/sample - loss: 1.0102 - accuracy: 0.4767 - val_loss: 0.9582 - val_accuracy: 0.4667
Epoch 4/10
86/86 [==============================] - 0s 399us/sample - loss: 1.0099 - accuracy: 0.4767 - val_loss: 0.9581 - val_accuracy: 0.4889
Epoch 5/10
86/86 [==============================] - 0s 296us/sample - loss: 1.0097 - accuracy: 0.4767 - val_loss: 0.9580 - val_accuracy: 0.4889
Epoch 6/10
86/86 [==============================] - 0s 251us/sample - loss: 1.0094 - accuracy: 0.4767 - val_loss: 0.9579 - val_accuracy: 0.4889
Epoch 7/10
86/86 [==============================] - 0s 276us/sample - loss: 1.0091 - accuracy:

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Reloading Oracle from /content/my_dir/Hyperband/oracle.json
INFO:tensorflow:Reloading Tuner from /content/my_dir/Hyperband/tuner0.json


INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Reloading Oracle from /content/my_dir/Hyperband/oracle.json
INFO:tensorflow:Reloading Tuner from /content/my_dir/Hyperband/tuner0.json


INFO:tensorflow:Oracle triggered exit


In [47]:
tuner_Hb.search_space_summary()


In [48]:
tuner_Hb.results_summary()

In [49]:
tuner_Hb.get_best_hyperparameters()

In [50]:
tuner_Hb.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

{'activation': 'hard_sigmoid',
 'drop_rate': 0.3,
 'learning_rate': 0.01,
 'tuner/bracket': 0,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 0,
 'tuner/round': 0,
 'units': 5,
 'weight_init': 'normal'}

In [0]:
best_model = tuner_Hb.get_best_models()[0]

##Plotting training and validation loss

In [0]:
import matplotlib.pyplot as plt

In [0]:
epochs = range(1, num_epochs+1)

In [54]:
plt.plot(epochs, average_loss_history, 'bo', label='training loss')
plt.plot(epochs, average_val_loss_history, 'b', label='validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

NameError: ignored

##Plotting train and validation accuracy

In [0]:
plt.plot(epochs, average_acc_history, 'bo', label='Training accuracy')
plt.plot(epochs, average_val_acc_history, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend() 

In [0]:
!pwd

In [0]:
!pip list